In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt

### Code to generate data is taken from:
https://github.com/pytorch/examples/tree/master/time_sequence_prediction

In [ ]:
np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L, 1), 'int64')
x[:,:,0] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = torch.tensor(np.sin(x / 1.0 / T).astype('float64')).type('torch.FloatTensor')#, device='cuda')
torch.save(data, open('traindata.pt', 'wb'))

print(data.shape)
#print(batches.shape)

#print(data[0,0])
#plt.plot(data[0])
#plt.plot(data[1])
plt.show()

def to_batches(sequence, batch_size):
    #print(sequence.size()[0])
    #print(sequence.size()[1])
    batches = sequence.view(sequence.size()[0], sequence.size()[1]//batch_size, batch_size, sequence.size()[2])
    return batches
    
batches = to_batches(data,1)
test = torch.tensor([[[1],[2],[3]],[[4],[5],[6]]])
print(test)
print(test.size())
test_batches = to_batches(test,3)
print(test_batches.size())
print(batches.size())

In [ ]:
simple = nn.LSTM(1, 1, 1)
#print(type(data[0]))
#print(len(data[0]))
temp = data[0]
#print(type(temp[0]))


test = [torch.Tensor([elem]) for elem in temp]

def convert_data (data):
    ret_val = []
    for i in data:
        temp = [torch.Tensor([elem]) for elem in i]
        ret_val.append(torch.cat(temp).view(len(temp),1,-1))
        
    return ret_val

#print(temp.size())
#simple.train(data)
h0 = torch.randn((1, 25, 1))#, device='cuda')
c0 = torch.randn((1, 25, 1))#, device='cuda')
epochs = 50
for j in range(epochs):
    for i in range(batches.size()[0]):
        output, (h0 , c0) = simple(batches[i], (h0, c0))

    
#output,_ = simple(batches[0], (h0,c0))
'''
converted_data = convert_data(data)
print(converted_data.size)
for elem in converted_data:
    output, (h0 , c0) = simple(elem, (h0, c0))
'''
print("output shape:", output.size())

#print("test",output[0].float())

#plt.scatter(range(1000),data[0])
#plt.scatter(range(1000), output[:].detach().numpy())
plt.plot(output.view(output.size()[0]*output.size()[1]).detach().numpy())
plt.show()
#plt.plot(output)
#simple.forward(torch.Tensor(test))


In [ ]:
lstm = nn.LSTM(1, 1)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 1) for _ in range(5)]  # make a sequence of length 5
print(inputs)

# initialize the hidden state.
hidden = (torch.randn(1, 1, 1),
          torch.randn(1, 1, 1))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
print(type(inputs))
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print(inputs.size())
hidden = (torch.randn(1, 1, 1), torch.randn(1, 1, 1))  # clean out hidden state
out, hidden = lstm(inputs, hidden)

print(out)
print(hidden)

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
    ]


word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

print(type(model.parameters()))
# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)